In [1]:
import os
import sys
import time
import numpy as np
import imgaug 

In [2]:
from mrcnn.config import Config
from mrcnn import model as modellib, utils

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils
from coco.coco import CocoDataset

Using TensorFlow backend.


In [3]:
COCO_PRETRAINED_WEIGHTS_PATH = os.path.abspath("mask_rcnn_coco.h5")
DATASET_PATH = os.path.abspath("DATASETS/")
TRAIN_ANNOTATIONS_PATH = os.path.abspath("DATASETS/annotations/instances_train2017.json")
LOGS_PATH = os.path.abspath("LOGS")

In [ ]:
class CocoConfig(Config):
    """Configuration for training on MS COCO.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    # Give the configuration a recognizable name
    NAME = "coco"
    
    IMAGES_PER_GPU = 1 #Only one image fits on my 6GB GTX1060. 
    #IMAGES_PER_GPU = 1 #version for Tesla P100
    
    NUM_CLASSES = 1 + 80  #80 COCO classes + BG class

In [ ]:
coco_train_annotatons = COCO(TRAIN_ANNOTATIONS_PATH)

loading annotations into memory...
Done (t=18.61s)
creating index...
index created!


In [ ]:
#Train dataset
dataset_train = CocoDataset()
dataset_train.load_coco(dataset_dir=DATASET_PATH, subset="train", year=2017, class_ids=None,
                        class_map=None, return_coco=False, auto_download=False)
dataset_train.prepare()



# Validation dataset
dataset_val = CocoDataset()
dataset_val.load_coco(dataset_dir=DATASET_PATH, subset="val", year=2017, auto_download=False)
dataset_val.prepare()

loading annotations into memory...
Done (t=18.46s)
creating index...
index created!
loading annotations into memory...
Done (t=0.52s)
creating index...
index created!


In [ ]:
# *** This training schedule is an example. Update to your needs ***

augmentation = imgaug.augmenters.Fliplr(0.5)

config = CocoConfig()

config.LEARNING_RATE = 0.001
config.LEARNING_MOMENTUM = 0.9

# Weight decay regularization
config.WEIGHT_DECAY = 0.0001

# Create model
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=LOGS_PATH)

# Training - Stage 1
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=1,
            layers='heads',
            augmentation=augmentation)

# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=120,
            layers='4+',
            augmentation=augmentation)

# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=160,
            layers='all',
            augmentation=augmentation)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/tomek/mask-rcnn-homework/LOGS/coco20190406T2034/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       

/home/tomek/anaconda3/envs/mask-rcnn/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/tomek/.local/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/tomek/anaconda3/envs/mask-rcnn/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/tomek/anaconda3/envs/mask-rcnn/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tomek/.local/lib/python3.6/site-packages/keras/utils/data_utils.py", line 666, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "/home/tomek/.local/lib/python3.6/site-packages/keras/utils/data_utils.py", line 661, in <lambda>
    initargs=(seqs, self.random_seed))
  File "/home/tomek/anaconda3/envs/mask-rcnn/lib/python3.6/multiprocessing/context.py", line 119, in Pool
    context=self.get_context())
  File "/home/tomek/anaconda3/envs/mask-rcnn/lib/python3.6/multiprocessing/pool.py", line 174, in __init__
    self._repopulate_pool()
  File "/home/tomek/anaconda3/envs/mask-rcnn/lib/python3.6/multiprocessing/pool.py", line 239,

In [ ]:
model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=args.logs)

# model = modellib.MaskRCNN(mode="inference", config=config,
#                           model_dir=args.logs)